# Sharelock Hoaw trust indicator

This script is used to build a score for a given document / post according to the 

### Preprocessing Data from Facebook

In [1]:
import sys
import time

reload(sys)
sys.setdefaultencoding('utf-8')

In [3]:
import newspaper

In [8]:
import facebook
import pandas as pd

# Token got with Facebook Developper
token = "EAACEdEose0cBAKh2BIdpbxBTzL4muuOAuefEKwdvdVZA8UVhI83ZC2gmFkr6ak5JuZAG6GTEpNPcJLyZA7YlBnsre1CYxvZC4QBXujseB0hW0PumhokNQJDZBeHBjaIvI2Ex7XkOolQhvG9luRe71SUHS7Up3T8ZBrkfB1nKVeVr18s5rTrO3Xx4aoMqPFBYq0ZD"

# Graph catched from Facebook. 
graph = facebook.GraphAPI(access_token=token, version='2.7')

In [9]:
file = "../data/facebook-fact-check.csv"
df = pd.read_csv(file, header=42, names=["account_id","post_id","Category","Page","Post URL","Date Published","Post Type","Rating","Debate","share_count","reaction_count","comment_count"])

In [10]:
df.head()

,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
0,184096565021911,1036268139804745,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-20,video,mostly true,NaN,231.0,572.0,76.0
1,184096565021911,1036296946468531,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-20,link,mostly true,NaN,31.0,170.0,97.0
2,184096565021911,1036336249797934,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-20,link,mostly true,NaN,NaN,13.0,4.0
3,184096565021911,1036354179796141,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-20,video,no factual content,NaN,NaN,39.0,6.0
4,184096565021911,1036370659794493,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-20,video,mostly true,NaN,104.0,627.0,206.0


In [11]:
# Get content from URL
test = graph.get_object(id=(str(df['account_id'][30]) + '_' + str(df['post_id'][30])))

In [12]:
test['message']

u'"Our purpose in this negotiation was to put an end to the kind of horrific and indiscriminate attacks that have been the primary cause of fear, of suffering, of displacement," John Kerry  said. "And under our plan, all of this could be quickly accompanied by serious negotiations between the parties aimed at a political transition and a conclusion to the conflict."'

In [14]:
df['Created-time'] = ['' for _ in range(len(df))]
df['Content'] = ['' for _ in range(len(df))]

In [26]:
start = time.time()
for i in range(1070,len(df["account_id"])):
    try:
        res = graph.get_object(id=(str(df['account_id'][i]) + '_' + str(df['post_id'][i])))
        df['Content'][i] = res['message'].decode('utf-8','ignore').encode("ascii",'ignore')
    except:
        pass
print "Done in %f s"%(time.time() - start)

In [28]:
len([x for x in df['Content'] if x])

2166

In [50]:
df.to_csv("../data/preprocessed_data.csv")

In [57]:
import csv

with open('../data/X_inputs.csv', 'wb') as myfile:
    wr = csv.writer(myfile, delimiter='\t', quoting=csv.QUOTE_ALL)
    wr.writerow(X_input)

In [58]:
with open('../data/y_inputs.csv', 'wb') as myfile:
    wr = csv.writer(myfile, delimiter='\t', quoting=csv.QUOTE_ALL)
    wr.writerow(y_input)

#### NO-NEED-TO-RELOAD POINT

In [4]:
import csv

# Load list
with open('../data/X_inputs.csv', 'r') as my_file:
    reader = csv.reader(my_file, delimiter='\t')
    X_input_load = list(reader)[0]

In [131]:
# Load list
with open('../data/y_inputs.csv', 'r') as my_file:
    reader = csv.reader(my_file, delimiter='\t')
    y_input_load = list(reader)[0]

In [132]:
y_input = [float(x) for x in y_input_load]

In [33]:
# Converting labels into real number between 0 and 1
range_lab = []
for i in range(len(df)):
    tmp = df['Rating'][i]
    if not(tmp in range_lab):
        range_lab.append(tmp)

In [34]:
range_lab

['mostly true',
 'no factual content',
 'mixture of true and false',
 'mostly false']

In [126]:
scores_label = dict()
scores_label['mostly true'] = 1.
scores_label['mixture of true and false'] = 0.5
scores_label['mostly false'] = 0.

In [63]:
import nltk
import string 
import os 

from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.stem.porter import PorterStemmer 

X_input = []
y_input = []

import re
pattern = "^([A-Z][0-9]+)+$"

from nltk.corpus import stopwords


def tokenize(text): 
    tokens = nltk.word_tokenize(text)
    stems = [] 
    for item in tokens:
        if (item not in stopwords.words('english')) and (item not in [',','"','.','`','``','*+',' ',"'","''",";"]):
            stems.append(PorterStemmer().stem(item.lower())) 
    return stems 

X_input = [tokenize(x) for x in X_input_load]

# for i in range(len(df)):
#     tmp = df['Content'][i]
#     rat = df['Rating'][i]
#     if (tmp and rat!='no factual content'):
#         X_input.append(' '.join(map(lambda x: tokenize(x),tmp.split(' '))))
#         y_input.append(scores_label[df['Rating'][i]])

In [69]:
X_input_str = [' '.join(x) for x in X_input]

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

no_features = 100

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(X_input_str)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(X_input_str)
tf_feature_names = tf_vectorizer.get_feature_names()

In [73]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [74]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

In [86]:
dict_word = dict()

for x in X_input:
    for tok in x:
        tmp = dict_word.get(tok,0)
        dict_word[tok] = tmp + 1

In [105]:
new_dict = dict()

for x in dict_word:
    tmp = dict_word[x]
    if tmp > 3:
        new_dict[x] = tmp

In [106]:
values = new_dict.values()
keys = new_dict.keys()

### LSA

In [116]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

In [137]:
lsa = TruncatedSVD(n_components=99)
X = lsa.fit_transform(tfidf)
X = Normalizer(copy=False).fit_transform(X)

In [138]:
len(X)

1949

### XGBOOST

In [139]:
import xgboost as xgb

In [140]:
# specify parameters via map
# param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
# num_round = 2
model = xgb.XGBRegressor()

model.fit(X,y_input)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [145]:
model.predict(X)

array([ 1.01550925,  0.95224071,  0.98753285, ...,  0.81422007,
        0.81422007,  0.81422007], dtype=float32)

In [148]:
min(model.predict(X))

0.15536425

### Testing

We here try to evaluate the quality of our model. We define a metric that discriminize the problem, and so where :
    - Good News are news with a score > 70%
    - A not-absolutely-true news has a score between 30% and 70%
    - Fake News have a score < 30%

In [149]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_input, test_size=0.20, random_state=42)

In [150]:
model = xgb.XGBRegressor()
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [151]:
def precision(label,score):
    if (int(2*label)==1 and score>=0.3 and score<=0.7):
        return 1
    elif (int(label==0) and score<=0.3):
        return 1
    elif (int(label)==1 and score>=0.7):
        return 1
    else:
        return 0

In [152]:
y_pred = model.predict(X_test)

In [153]:
score = 0
tot = 0
for true,pred in zip(y_test,y_pred):
    tot += 1
    score += precision(true,pred)

In [155]:
score, tot

(330, 390)

In [162]:
# Score of well_detected/total_posts
float(score)/float(tot)

0.8461538461538461